### Reference
https://github.com/AlexeyAB/darknet/

### Folder Structure and Explanation
>       drive/
        └── MyDrive/
                └── project/
                    └── darknet/
                        ├── cfg/
                        │   ├── yolov4.conv.137 ---> initial weights file
                        │   ├── yolov4-custom.cfg ---> initial cfg file
                        │   └── yolov4-obj.cfg ---> modified cfg file for training
                        ├── train/ ---> this folder is created to stor training dataset/
                        │   ├── obj/
                        │   │   ├── frame_0001.PNG ---> training image
                        │   │   └── frame_0001.txt ---> coordation of bounding box
                        │   ├── obj.data ---> path to each dataset
                        │   ├── obj.names ---> label names
                        │   ├── train.txt ---> path to training image
                        │   └── valid.txt ---> path to evaluation image
                        └── backup/ ---> this folder is created to save trained weights file/
                            ├── yolov4-obj_last.weights
                            ├── yolov4-obj_1000.weights
                            └── yolov4-obj_best.weights

### Before you run the code, please chage the runtime type to GPU
When you change the runtime type current directory is changed to root direcotry (content dir)

# Set up Darknet

### Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive

### Create new directory for training

In [ ]:
%mkdir project

In [ ]:
%cd project

### Clone the darknet from the github

In [ ]:
!git clone https://github.com/AlexeyAB/darknet/

In [ ]:
%cd darknet

### Change Makefile
OPENCV=1
GPU=1
CUDNN_HALF=1
CUDNN=1

In [ ]:
%cat Makefile

In [16]:
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile

### Install daknet

In [ ]:
!make

# Setup Yolo Training with Darknet

### Download pre-trained weights

In [ ]:
%cd cfg

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

### Create custom yolov4.cfg file

In [21]:
%cp yolov4-custom.cfg yolov4-obj.cfg

Open this link than edit file as follows

/content/drive/MyDrive/project/darknet/cfg/yolov4-obj.cfg

1. change line batch to batch=64
2. change line subdivisions to subdivisions=16
3. change width and height to 416 (or any value multiple of 32)
4. change line max_batches to (classes*2000, but not less than number of training images and not less than 6000) 
5. change line steps to 80% and 90% of max_batches
6. change line classes=80 to your number of objects in each of 3 [yolo]-layers:
7. change [filters=255] to filters=(classes + 5)x3 in the 3 [convolutional] before each [yolo] layer, keep in mind that it only has to be the last [convolutional] before each of the [yolo] layers.


for our project we have 3 objects(Halibut, Trap, Cod)
- max_batches=6000
- steps=4800,5400
- classes=3 (line: 970, 1058, 1146)
- filters=24 (line: 963, 1051, 1139)

In [35]:
!cd ..

### Create new directory in the darknet folder and upload the dataset into it.

In [ ]:
%mkdir train

### If you using cvat you need to delete unlabeled image and split data for training and validation.
- download "delete_empty_data.py and "split_data.py" from Archipelago-Object-Tracking github repository and place them under darknet folder.

Enter directory path to data file to delete empty(unlabeled) data.

In [ ]:
!python delete_empty_data.py train/obj/

Enter directory path to data file to split the dataset.
It will create train.txt and valid.txt under train.

In [ ]:
!python split_data.py train/obj/

### Create new directory to save weights file

In [ ]:
%mkdir backup

### Open obj.data file and change the filepath
- classes = 3    (number of labels)
- train = train/train.txt    (dataset for training)
- valid = train/valid.txt    (dataset for evaluation)
- names = train/obj.names    (list of label)
- backup = backup/    (where weights file will be saved)

# Train Yolo with Darknet

### for permission error
you will get permission error when you reconnect the google colab.

In [6]:
!chmod +x ./darknet

### First time to train yolo
- darknet will save every 100 iteraions to '_last.weights'
- darknet will create another weights file every 1000 iterations with file name '_1000.weights'
- after 1000 iterations, it will create '_best.weights' file which has best mAP(mean average precision)
- so if you want to save 1500 iterations weights file, you should save it manully.


Arguments
- detector
- train : type of process
- train/obj.data : path to the training data
- cfg/yolov4-obj.cfg : path to the cfg file
- cfg/yolov4.conv.137 : path to the weight file
- -don't_show : does not show the graphical process window
- -map : do the evaluation during the training

In [ ]:
!./darknet detector train train/obj.data cfg/yolov4-obj.cfg cfg/yolov4.conv.137 -dont_show -map

### Resume train with previous weights file

In [ ]:
!./darknet detector train train/obj.data cfg/yolov4-obj.cfg backup/yolov4-obj_best.weights -dont_show -map

### for validation

In [ ]:
!./darknet detector valid train/obj.data cfg/yolov4-obj.cfg backup/yolov4-obj_1000.weights -dont_show -map

to get precision

In [ ]:
!./darknet detector map train/obj.data cfg/yolov4-obj.cfg backup/yolov4-obj_1000.weights -dont_show

### for testing

In [ ]:
!./darknet detector test train/obj.data cfg/yolov4-obj.cfg backup/yolov4-obj_1000.weights -dont_show

### get output with darknet

In [ ]:
!./darknet detector demo train/obj.data cfg/yolov4-obj.cfg backup/yolov4-obj_best.weights -dont_show video/fish.avi -i 0 -out_filename video/output.avi